# RCS Benchmark

## Getting Started

You need [`pyqrack`](https://pypi.org/project/pyqrack/) to run this notebook.

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

In the Python package itself, there should be an executable called `qrack_cl_precompile`. This "pre-compiles" the OpenCL "just-in-time" ("JIT") device program, for your system's accelerators. You might want to find and run this utility first, to avoid the need to "recompile" the OpenCL device program every time you first load Qrack into your environment.

## Configuration

In [2]:
# Lowest qubit width
low = 8
# Highest qubit width
high = 30
# Samples to average per qubit width
samples = 1

In [3]:
import math
import os
import random
import time

from pyqrack import QrackSimulator

# For more specific details about all available Qrack environment variables,
# See the C++ repository README: https://github.com/unitaryfund/qrack

# "Device ID" is the sequential index number output for each accelerator,
# in the "banner" whenever Qrack is loaded with GPU build options.
# This is the "device ID" for your primary or main (single) accelerator.
os.environ['QRACK_OCL_DEFAULT_DEVICE']='0'
# If you have multiple accelerators, "QUnitMulti" will attempt to distribute
# completely separable subsystems, when they arise, to multiple separate devices.
# Use this variable to input a comma-separated list of devices for "QUnitMulti."
os.environ['QRACK_QUNITMULTI_DEVICES']='0'
# If you have multiple accelerators, "QPager" can distribute (entangled) simulations
# across multiple equal-sized "pages" of state vector amplitudes.
# Use this variable to input a comma-separated list of device-to-"page" mappings.
os.environ['QRACK_QPAGER_DEVICES']='0'
# Some accelerators, like Intel integrated graphics, actually use general system RAM.
# In this case, OpenCL can be told to allocate on general "host" instead of "device" RAM.
# For each entry above in 'QRACK_QPAGER_DEVICES', below, "1" means "host," "0" means "device."
os.environ['QRACK_QPAGER_DEVICES_HOST_POINTER']='0'
# This is the maximum qubit count you want to fit in a GPU "maximum allocation segment."
# (Your GPU probably has 4 such segments. You might want this 1 less than theoretical max,
# so that "memory fragmentation" doesn't prevent using more than 1 segment in total.)
os.environ['QRACK_MAX_PAGE_QB']='27'
# This is an overall allocation limit for your GPU(s), in megabytes.
# If you have multiple GPUs, you can list separate limits in device ID order,
# separated by a comma.
os.environ['QRACK_MAX_ALLOC_MB']='15872'
# This is the maximum total number of fully-entangled qubits you expect to achieve using QPager.
os.environ['QRACK_MAX_PAGING_QB']='30'
# This is the maximum total number of fully-entangled qubits you expect to fit in general RAM.
os.environ['QRACK_MAX_CPU_QB']='32'
# Above this threshold, "QTensorNetwork" restricts simulations to "past light cone."
# At or below the threshold, much more work can be reused.
os.environ['QRACK_QTENSORNETWORK_THRESHOLD_QB']='30'

# These below are approximation options. (By default, Qrack simulates in the "ideal.")

# This is a number between "0" ("ideal") and "1" ("destroy all entanglement") for "SDRP,"
# "Schmidt decomposition rounding parameter". (https://arxiv.org/abs/2304.14969)
# os.environ['QRACK_QUNIT_SEPARABILITY_THRESHOLD']='0.5'

# This is a number between "0" ("ideal") and "0.5" ("combine all binary decision tree branches")
# that sets the allowable "epsilon" between "QBdt" branches to consider them equal.
# os.environ['QRACK_QBDT_SEPARABILITY_THRESHOLD']='0.1'

## Run the Benchmark

In [4]:
def bench_qrack(n):
    # This is basically a "quantum volume" (random) circuit.
    start = time.perf_counter()

    sim = QrackSimulator(n)

    lcv_range = range(n)
    all_bits = list(lcv_range)

    single_count = 0
    double_count = 0
    for _ in lcv_range:
        # Single-qubit gates
        for i in lcv_range:
            sim.u(i, random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi))

        # 2-qubit couplers
        unused_bits = all_bits.copy()
        random.shuffle(unused_bits)
        while len(unused_bits) > 1:
            sim.mcx([unused_bits.pop()], unused_bits.pop())

    # Terminal measurement
    # print(sim.m_all())
    sim.m_all()

    return time.perf_counter() - start

# Make sure the OpenCL environment is initialized before timing.
# (You probably also want to precompile OpenCL kernels with the `qrack_cl_compile` utility, in general.)
bench_qrack(1)

qrack_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_qrack(n))

    qrack_results[n] = sum(width_results) / samples
    print(n, ": ", qrack_results[n], " seconds")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
8 :  0.0025180409975291695  seconds
9 :  0.0031449689995497465  seconds
10 :  0.005791810999653535  seconds
11 :  0.012204154998471495  seconds
12 :  0.01745402400047169  seconds
13 :  0.019789517002209323  seconds
14 :  0.0243603160015482  seconds
15 :  0.024280266999994637  seconds
16 :  0.03236640300019644  seconds
17 :  0.04010078999999678  seconds
18 :  0.040536922999308445  seconds
19 :  0.05422422700212337  seconds
20 :  0.12229957100134925  seconds
21 :  0.21900078499675146  seconds
22 :  0.42920195199985756  seconds
23 :  0.8274721360030526  seconds
24 :  1.7695226830001047  seconds
25 :  3.577587391999259  seconds
26 :  7.846220686999004  seconds
27 :  16.53956957799892  seconds
28 :  38.96062470600009  seconds
29 :  81.3354682049976  seconds
30 :  178.05179994599894  seconds
